In [1]:
import ipywidgets as widgets

import rospy

from geometry_msgs.msg import PoseStamped
from assignment2_rt.msg import PlanningActionGoal
from assignment2_rt.srv import Target, TargetResponse

In [2]:
last_target = None
service_client = None
out, out_service = widgets.Output(), widgets.Output()

In [3]:
display(out_service)

Output()

In [4]:
def goal_callback(msg):
    global last_target
    last_target = msg

def handle_last_goal_request(req):
    global last_target, out
    if last_target is None:
        return TargetResponse()
    target_info = PoseStamped()
    target_info.header = last_target.header
    target_info.pose = last_target.goal.target_pose.pose
    return TargetResponse(target_info)    

def service_last_goal():
    global out_service, service_client
    rospy.init_node('retrieve_last_target')
    rospy.Subscriber('/reaching_goal/goal', PlanningActionGoal, goal_callback)
    rospy.Service('/get_last_goal', Target, handle_last_goal_request)
    with out_service:
        print("[SERVICE] Service node started. Waiting for requests...")
        
    service_client = rospy.ServiceProxy('/get_last_goal', Target)
    with out_service:
        print("[SERVICE] Service proxy for calls started")

try:
    service_last_goal()
except rospy.ROSInterruptException:
    global out_service
    with out_service:
        print("Program interrupted")

In [5]:
request_button = widgets.Button(description="Retrieve last goal")
display(request_button)

def on_button_clicked(b):
    global service_client, out
    out.clear_output()
    try:
        resp = service_client()
        with out:
            if resp.last_target.header.seq != 0:
                print(f"Response received:\n {resp}")
            else:
                print("No target available")
    except rospy.ServiceException as e:
        with out: 
            print(f"Service call failed: {e}")

request_button.on_click(on_button_clicked)

Button(description='Retrieve last goal', style=ButtonStyle())

In [6]:
display(out)

Output()